# Hybrid Retriever - Combining Sparse and Dense retrievers

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_classic.schema import Document

e:\Babu\ragudemy\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# sample documents
docs = [
    Document(page_content="Langchain helps build LLM applications"),
    Document(page_content="Pinecone is a vector database for semantic search"),
    Document(page_content="The Eiffel tower is located in Paris"),
    Document(page_content="Langchain can be used to develop agentic ai applications"),
    Document(page_content="Langchain has many types of retrievers"),
]

# dense retiever (FAISS + Huggingface)
embedding_model = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

In [3]:
# Sparse retriever - BM25
sparse_retriever = BM25Retriever.from_documents(docs)
sparse_retriever.k=3 # top k documents to retrieve

# combine with ensember retriever 
hybrid_retriever = EnsembleRetriever(
    retrievers = [dense_retriever, sparse_retriever],
    weights=[0.7,0.3]
)

In [4]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B22AA3F940>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001B229613100>, k=3)], weights=[0.7, 0.3])

In [5]:
# query and get results
query = "How can I build an application using LLMs?"
results = hybrid_retriever.invoke(query)

In [6]:
# print results
for i, doc in enumerate(results):
    print(f"\n Document {i+1}:\n{doc.page_content}")


 Document 1:
Langchain helps build LLM applications

 Document 2:
Langchain can be used to develop agentic ai applications

 Document 3:
Langchain has many types of retrievers

 Document 4:
Pinecone is a vector database for semantic search


# RAG pipeline with hybrid retriever

In [10]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

In [9]:
# Prompt template
prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}
                                      
Question: {input}
""")

In [24]:
# LLM
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = init_chat_model(model="openai:gpt-5-nano-2025-08-07",
                      temperature = 0.2)
llm

ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x000001B2710FFF10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B2710FC310>, root_client=<openai.OpenAI object at 0x000001B2710FD210>, root_async_client=<openai.AsyncOpenAI object at 0x000001B2710FCDC0>, model_name='gpt-5-nano-2025-08-07', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [25]:
# create stuff document chain

document_chain = create_stuff_documents_chain(
    prompt=prompt,
    llm=llm
)

In [26]:
# create full rag chain
rag_chain = create_retrieval_chain(
    retriever=hybrid_retriever,
    combine_docs_chain=document_chain
)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B22AA3F940>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001B229613100>, k=3)], weights=[0.7, 0.3]), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context below.\n\nContext:\n{context}\n                                      \nQuestion: {input

In [27]:
# ask a question
query = {"input": "How can I build a app using LLMs?"}
response = rag_chain.invoke(query)

print(f"Answer \n {response['answer']}")

Answer 
 Here’s a practical way to build an app that uses large language models (LLMs), leveraging LangChain and a vector database like Pinecone.

What you’ll build a lot of times
- A retrieval-augmented generator: user asks a question, the app fetches relevant documents, and the LLM generates an answer with references.
- An agentic AI: the LLM can take actions (tools, APIs, or custom functions) to fulfill tasks and fetch data.

Recommended workflow

1) Define the app and data
- Clarify the user goal (e.g., document Q&A, customer support, code helper, research assistant).
- Gather the data you’ll search over (internal docs, PDFs, PDFs, PDFs, web scraped content, etc.).

2) Pick LLM and hosting
- Choose an LLM provider (e.g., OpenAI, Cohere, or others) and set up access.
- Decide how you’ll host/run your app (local server, cloud function, or a web app).

3) Set up LangChain and your vector store
- Install LangChain and the Pinecone client.
- Create embeddings for your documents (embeddi

In [28]:
print(f"\n Source Documents:")
for i, doc in enumerate(response['context']):
    print(f"\nDoc {i+1}: {doc.page_content}")


 Source Documents:

Doc 1: Langchain helps build LLM applications

Doc 2: Langchain can be used to develop agentic ai applications

Doc 3: Pinecone is a vector database for semantic search

Doc 4: Langchain has many types of retrievers
